In [ ]:
import csv
import nytimesarticle
from nytimesarticle import articleAPI
api = articleAPI('943d65691e814b8b96e1193dcab0036e')


def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    if 'response' in articles :
        for i in articles["response"]["docs"]:
            dic = {}
            dic['id'] = i['_id']

            dic['headline'] = i['headline']['main'].encode("utf8")

            dic['date'] = i['pub_date'][0:10]

            dic['source'] = i['source']
            dic['type'] = i['type_of_material']
            dic['url'] = i['web_url']
            dic['word_count'] = i['word_count']
            news.append(dic)
    return(news) 


all_articles = []
for i in range(0,100):
    articles = api.search(q = "marketing",
           begin_date = 20180301,
           end_date = 20180315,
           page = i)
    articles = parse_articles(articles)
    all_articles = all_articles + articles
    
    
import csv
keys = all_articles[0].keys()
outputfile = open('NewsData_3.csv', 'w',newline='') 
dict_writer = csv.DictWriter(outputfile,keys)
dict_writer.writerows(all_articles)
outputfile.close()

In [3]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import csv


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


i=1
with open ('NewsData.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        html = urllib.request.urlopen(row[5]).read()
        outputfile = open("NYT%s.txt" %i,"w")
        i+=1
        outputfile.write(text_from_html(html))
        outputfile.close()